In [3]:
from transformers import GPT2Model, GPT2Tokenizer

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
# Modeling
# Initialize sentence transformer
sentence_transformer = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2').to(device)

# Initialize Faiss index
dim = sentence_transformer.get_sentence_embedding_dimension()
index = faiss.IndexFlatIP(dim)

# Initialize GPT-2 model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

encoder = model.transformer.wte.to(device) # access the encoder (word token embeddings)
decoder = model.lm_head.to(device) # access the decoder (language model head)

In [ ]:
# Inference
def inference(model, query):
    # Encode query
    query_emb = sentence_transformer.encode(query)

    # Retrieve top 10 documents
    scores, indices = index.search(np.vstack(doc_embs), 10)
    top_docs = [doc_embs[idx] for idx in indices[0]]

    # Concatenate query and top docs
    input_emb = torch.cat([query_emb] + top_docs, dim=0)

    # Encode input
    encoded = model.encoder(input_emb)

    # Generate answer
    output = model.decoder.generate(encoded, max_length=100)

    # Decode answer
    answer = tokenizer.decode(output[0], skip_special_tokens=True)

    return answer


# Define model
class QAModel(nn.Module):
    def __init__(self, encoder, decoder):
        super(QAModel, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, query, docs, answer):
        # Encode query and docs
        print(f"\nquery shape is {query.shape}")
        print(f"\ndocs shape is {docs.shape}")
        print(f"\nanswer shape is {answer.shape}")

        query_emb = torch.tensor(sentence_transformer.encode(query)).to(device)
        print(f"\nquery_emb shape is {query_emb.shape}")

        doc_embs = [torch.tensor(sentence_transformer.encode(doc)).to(device) for doc in docs]
        print(f"\ndoc_embs shape is {doc_embs.shape}")

        # Retrieve top 10 documents
        top_k = 10
        scores, indices = index.search(np.vstack(doc_embs), top_k)
        top_docs = [doc_embs[idx] for idx in indices[0]]

        # Concatenate query and top docs
        input_emb = torch.cat([query_emb] + top_docs, dim=0)

        # Encode input
        encoded = self.encoder(input_emb)

        # Decode answer
        output = self.decoder(encoded, answer)

        return output
    
model = QAModel(encoder, decoder)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
num_epochs = 1

# Training loop
for epoch in range(num_epochs):
    for query, answer, docs in tqdm(dataloader):
        # Convert query, answer to tensors if not already (adjust based on actual data format)
        if not isinstance(query, torch.Tensor):
            query = torch.tensor(query)  # Add dtype and device as needed, e.g., dtype=torch.long, device=device
        if not isinstance(answer, torch.Tensor):
            answer = torch.tensor(answer)  # Add dtype and device as needed

        query, answer = query.to(device), answer.to(device)
        # Ensure docs is a list of tensors, adjust as necessary based on your actual data
        docs = [torch.tensor(doc).to(device) if not isinstance(doc, torch.Tensor) else doc.to(device) for doc in docs]

        optimizer.zero_grad()
        output = model(query, docs, answer)
        loss = criterion(output, answer)
        loss.backward()
        optimizer.step()